In [ ]:
#Create and evaluate forward models

In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
#read tm data and split it
resmat = pd.read_excel("tm_results_expanded.xlsx")
X = resmat.iloc[:,:4]*(10**9)
Y = resmat.iloc[:,11:]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42);
X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(X, Y, random_state=None);
X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(X, Y, random_state=None);
X_train_3, X_test_3, Y_train_3, Y_test_3 = train_test_split(X, Y, random_state=None);
X_train_4, X_test_4, Y_train_4, Y_test_4 = train_test_split(X, Y, random_state=None);

In [3]:
#scale train and test data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_test_1 = scaler.transform(X_test_1)
X_test_2 = scaler.transform(X_test_2)
X_test_3 = scaler.transform(X_test_3)
X_test_4 = scaler.transform(X_test_4)
r = np.amax(X_train, axis = 0) - np.amin(X_train, axis = 0)

In [4]:
#train reference model - 3 hidden layers of 100 neurons each, 1000 iterations
nn_ref = MLPRegressor(solver="lbfgs", max_iter=1000, alpha=0.05, hidden_layer_sizes=(100,100,100))
nn_ref.fit(X_train, Y_train)

MLPRegressor(alpha=0.05, hidden_layer_sizes=(100, 100, 100), max_iter=1000,
             solver='lbfgs')

In [5]:
#evaluate reference model
ref_score = [nn_ref.score(X_test, Y_test), nn_ref.score(X_test_1, Y_test_1), nn_ref.score(X_test_2, Y_test_2),
            nn_ref.score(X_test_3, Y_test_3), nn_ref.score(X_test_4, Y_test_4)]
print(np.average(ref_score))
print(ref_score)

0.9234166266910083
[0.9171038198413188, 0.923666538749603, 0.9196428673035602, 0.9288482865762111, 0.9278216209843491]


In [7]:
#creates a list of neuron layers build
def hls_list(min_l=2, max_l=5, step_l=1, min_n=100, max_n=500, step_n=100):
    hls = list()
    for layer in np.arange(min_l, max_l+1, step_l):
        for neuron in np.arange(min_n, max_n+1, step_n):
            tup = tuple()
            for i in range(layer):
                tup = tup + (neuron,)
            hls.append(tup)
    return hls

In [16]:
#performing grid search to avoid overfitting - try 1
hls = hls_list(1,2,1,50,200,50)
m = [500, 1000, 1500, 2000]

par = {'hidden_layer_sizes':hls, 'max_iter':m}

nnl = MLPRegressor(solver="lbfgs", alpha=0.05)
regl_1 = GridSearchCV(nnl, par, scoring='r2', cv=5)
regl_1.fit(X_train,Y_train)
print("try 1:")
print(regl_1.cv_results_)
print(regl_1.best_estimator_)
print(regl_1.best_score_)
print(regl_1.best_params_)

/Users/saarpop/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/saarpop/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/saarpop/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to c

try 1:
{'mean_fit_time': array([ 0.22133904,  0.23356776,  0.23279657,  0.20459447,  0.24038587,
        0.21025872,  0.23469787,  0.22874274,  0.32195392,  0.33506532,
        0.30285139,  0.2941442 ,  0.38873997,  0.38968735,  0.3754375 ,
        0.38234634,  0.46606936,  0.48500967,  0.49843531,  0.79468136,
        1.58221903,  1.93184328,  1.65714383,  2.10563693,  4.22151041,
        7.01039491,  9.24255137,  9.03369508, 10.8866044 , 16.4134809 ,
       14.8168252 ,  8.88196821]), 'std_fit_time': array([0.02601463, 0.02623877, 0.03405645, 0.04569359, 0.0366831 ,
       0.06482824, 0.04585067, 0.04128138, 0.0371722 , 0.02507235,
       0.04492881, 0.02305972, 0.05819073, 0.06803271, 0.07314919,
       0.03677966, 0.06302396, 0.04513261, 0.06414534, 0.13228924,
       0.25069388, 0.18967239, 0.09161909, 0.13878529, 0.6333892 ,
       2.48376286, 1.43631093, 1.96257518, 1.5529316 , 4.69925907,
       3.88614807, 2.04166678]), 'mean_score_time': array([0.00199599, 0.00102463, 0.00088

In [17]:
#performing grid search to avoid overfitting - try 2
hls = hls_list(1,3,1,50,200,50)
m = [2000, 2500, 3000]

par = {'hidden_layer_sizes':hls, 'max_iter':m}

nnl = MLPRegressor(solver="lbfgs", alpha=0.05)
regl_2 = GridSearchCV(nnl, par, scoring='r2', cv=5)
regl_2.fit(X_train,Y_train)
print("try 2:")
print(regl_2.cv_results_)
print(regl_2.best_estimator_)
print(regl_2.best_score_)
print(regl_2.best_params_)

try 2:
{'mean_fit_time': array([ 0.19075189,  0.21492414,  0.25533504,  0.31385059,  0.30694718,
        0.32699142,  0.3445509 ,  0.36271157,  0.38317938,  0.41351047,
        0.41940584,  0.43773594,  0.51108418,  0.51479802,  0.61761322,
        1.82380843,  1.84998083,  2.14473424,  4.73680606,  4.69504719,
        4.35755715,  8.6428555 ,  8.89828992,  9.1624011 ,  1.33106461,
        1.40099516,  1.39297032,  4.43224883,  5.09260068,  5.88536863,
       12.87838459, 13.59945173,  8.07410212, 22.49426022, 22.3710556 ,
       28.70034227]), 'std_fit_time': array([ 0.01926635,  0.02977282,  0.04862976,  0.06764048,  0.05992984,
        0.0179921 ,  0.03226616,  0.04354156,  0.02194926,  0.0393813 ,
        0.04307878,  0.08307712,  0.07987084,  0.08331658,  0.12981845,
        0.11222486,  0.18401399,  0.47330171,  0.66713664,  0.9825225 ,
        0.4087837 ,  1.3767522 ,  0.60119158,  2.01584654,  0.33962212,
        0.25345757,  0.23037378,  0.60214681,  1.04496748,  1.45311608,
 

In [18]:
#performing grid search to avoid overfitting - try 3
hls = hls_list(1,2,1,200,400,100)
m = [2000, 2500, 3000]

par = {'hidden_layer_sizes':hls, 'max_iter':m}

nnl = MLPRegressor(solver="lbfgs", alpha=0.05)
regl_3 = GridSearchCV(nnl, par, scoring='r2', cv=5)
regl_3.fit(X_train,Y_train)
print("try 3:")
print(regl_3.cv_results_)
print(regl_3.best_estimator_)
print(regl_3.best_score_)
print(regl_3.best_params_)

try 3:
{'mean_fit_time': array([ 0.49231176,  0.39713178,  0.34978275,  0.55237021,  0.58515449,
        0.57107129,  0.60924716,  0.67413664,  0.57692442,  6.84934063,
        6.8922173 ,  6.70762777, 13.55081224, 29.80652933, 30.87259512,
       57.5959197 , 56.19969816, 65.51237946]), 'std_fit_time': array([ 0.05736118,  0.0610289 ,  0.0708945 ,  0.0909616 ,  0.07310835,
        0.10458827,  0.13396647,  0.08067241,  0.21145845,  1.98303831,
        1.28510412,  2.14773607,  4.50496255, 12.96575018,  7.95617538,
       19.12729821, 19.51957047, 33.67731493]), 'mean_score_time': array([0.00150805, 0.00103083, 0.00098562, 0.00133457, 0.00102725,
       0.00135918, 0.00159545, 0.0016326 , 0.00108261, 0.00223246,
       0.00181494, 0.00219908, 0.00309629, 0.00646653, 0.00546517,
       0.00677705, 0.00907745, 0.00871639]), 'std_score_time': array([6.71277972e-04, 8.90869900e-05, 1.20424467e-04, 2.90454880e-04,
       4.28937752e-05, 2.97485989e-04, 6.89528862e-04, 5.82057691e-04,
      

In [19]:
#Check best model
nn_gs_1 = regl_1.best_estimator_
nn_gs_2 = regl_2.best_estimator_
nn_gs_3 = regl_3.best_estimator_
gs_1_score = [nn_gs_1.score(X_test, Y_test), nn_gs_1.score(X_test_1, Y_test_1), nn_gs_1.score(X_test_2, Y_test_2),
            nn_gs_1.score(X_test_3, Y_test_3), nn_gs_1.score(X_test_4, Y_test_4)]
gs_2_score = [nn_gs_2.score(X_test, Y_test), nn_gs_2.score(X_test_1, Y_test_1), nn_gs_2.score(X_test_2, Y_test_2),
            nn_gs_2.score(X_test_3, Y_test_3), nn_gs_2.score(X_test_4, Y_test_4)]
gs_3_score = [nn_gs_3.score(X_test, Y_test), nn_gs_3.score(X_test_1, Y_test_1), nn_gs_3.score(X_test_2, Y_test_2),
            nn_gs_3.score(X_test_3, Y_test_3), nn_gs_3.score(X_test_4, Y_test_4)]
print(gs_1_score)
print(gs_2_score)
print(gs_3_score)

[0.9657626705776499, 0.9708611956013862, 0.9704482170447103, 0.970598094251965, 0.9691581573471839]
[0.9675536593070759, 0.9737266078250628, 0.9721281533807484, 0.9734762293299448, 0.971367854610234]
[0.965638717164079, 0.969670342430768, 0.968309801503918, 0.9683972011310087, 0.9652074554537547]


In [ ]:
from matplotlib import pyplot
train_scores, test_scores = list(), list()
for al in a:
    nnl = MLPRegressor(solver="lbfgs", max_iter=2000, hidden_layer_sizes=al, alpha=0.05)
    nnl.fit(X_train,Y_train)
    # evaluate on the train dataset
    train_yhat = nnl.predict(X_train)
    train_acc = r2_score(Y_train, train_yhat)
    train_scores.append(train_acc)
    # evaluate on the test dataset
    test_yhat = nnl.predict(X_test)
    test_acc = r2_score(Y_test, test_yhat)
    test_scores.append(test_acc)
    # summarize progress
    print('>%d, train: %.3f, test: %.3f' % (al[0], train_acc, test_acc))

# plot of train and test scores vs tree depth
pyplot.plot(b, train_scores, '-o', label='Train')
pyplot.plot(b, test_scores, '-o', label='Test')
pyplot.legend()
#pyplot.xscale('log')
pyplot.show()

In [ ]:
print(scaler.inverse_transform(X_test), nn.predict(X_test))
print(scaler.inverse_transform(X_train), nn.predict(X_train), Y_train)

In [ ]:
#Create gradient descent process and find optimal geometry

In [8]:
#vector absolute value
def vecabs(a):
    s = 0
    for el in a:
        s = s + pow(el,2)
    return np.sqrt(s)

In [9]:
#loss calculation
def loss(s, est ,g, mu_g, rg):
    sp = est.predict([g])[0];
    return (pow((sp[0]-s[0]),2) + pow((sp[1]-s[1]), 2))
''' + relu(abs(g[0]-mu_g[0]) - 1/2*rg[0]) + 
            relu(abs(g[1]-mu_g[1]) - 1/2*rg[1]) + relu(abs(g[2]-mu_g[2]) - 1/2*rg[2]) + 
            relu(abs(g[3]-mu_g[3]) - 1/2*rg[3]))'''

' + relu(abs(g[0]-mu_g[0]) - 1/2*rg[0]) + \n            relu(abs(g[1]-mu_g[1]) - 1/2*rg[1]) + relu(abs(g[2]-mu_g[2]) - 1/2*rg[2]) + \n            relu(abs(g[3]-mu_g[3]) - 1/2*rg[3]))'

In [10]:
#ReLU
def relu(x):
    return np.maximum(0,x)

In [11]:
#calculate single iteration of gradient descent adjustment
def grad(s, est, g, delta, mu_g, rg):
    del0 = np.array([delta[0], 0, 0, 0])
    del1 = np.array([0, delta[1], 0, 0])
    del2 = np.array([0, 0, delta[2], 0])
    del3 = np.array([0, 0, 0, delta[3]])
    del_l0 = loss(s, est, g + del0, mu_g, rg) - loss(s, est, g, mu_g, rg)
    del_l1 = loss(s, est, g + del1, mu_g, rg) - loss(s, est, g, mu_g, rg)
    del_l2 = loss(s, est, g + del2, mu_g, rg) - loss(s, est, g, mu_g, rg)
    del_l3 = loss(s, est, g + del3, mu_g, rg) - loss(s, est, g, mu_g, rg)
    return [del_l0/delta[0], del_l1/delta[1], del_l2/delta[2], del_l3/delta[3]]

In [12]:
#Find optimal g given initial g
def opt(s, est, g_ini, delta, lam, mu_g, rg, tol, max_iter):
    g_hat = g_ini
    prev = 0
    curr = loss(s, est, g_hat, mu_g, rg)
    count = 0
    while(abs(curr-prev) > tol and count < max_iter):
        prev = curr
        g_hat = g_hat - lam*np.array(grad(s, est, g_hat, delta, mu_g, rg))
        curr = loss(s, est, g_hat, mu_g, rg)
        count += 1
    print(count)
    return g_hat

In [13]:
#Perform GD on ref model
nn = nn_ref
mu_g = np.mean(X_train, axis = 0)
delta = [1/10000, 1/10000, 1/10000, 1/2000]
g_min = X_train[0]
loss_min = 1
for x in X_train:
    g = opt([0,1], nn, x, delta, 0.01, mu_g, r, 1e-5, 1e3)
    l = loss([0,1], nn, g, mu_g, r)
    if (l < loss_min):
        g_min = g
        loss_min = l
print(loss_min)
nn.predict([g_min])

1000
1000
1000
1000
1000
1
594
975
690
490
1000
1000
1000
151
1000
1000
1000
1000
479
806
987
213
1000
506
933
1000
1000
1000
378
1000
583
935
930
1000
1000
931
333
653
584
1000
1000
1000
1000
812
989
1000
1000
950
1000
1000
1000
1000
249
1000
48
1000
424
1000
1000
1000
1000
1000
1000
244
1000
1000
1000
1000
431
574
1000
1
1000
1000
1000
1000
1000
1000
1000
1
309
1000
1000
1000
701
1000
1000
1
1000
1000
231
1000
54
644
1000
1000
169
1000
1000
1000
669
837
1000
517
694
1000
509
445
1000
1000
726
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
531
1000
1000
460
437
1000
897
1000
178
1000
647
1000
589
1
1000
1000
618
500
1000
1000
1000
752
1000
656
1000
1000
1000
1000
1000
1000
1
1000
1
1000
1000
1000
789
549
659
1000
1000
1000
1000
1000
1000
1000
794
1000
154
1000
501
157
1000
1
1
342
1000
812
1000
1000
248
1
1000
1000
1000
604
1000
1
1000
1000
1000
1000
1000
995
789
1000
279
1000
1000
1000
770
1000
880
1000
805
1000
1000
1000
1000
389
1000
1
768
798
130
524
1000
1000
1000
1000
68

array([[0.01913309, 0.98825536]])

In [14]:
#Present optimal reference geometry
g_min_ref = g_min
print("The optimal geometry, according to AI magic:", scaler.inverse_transform(g_min_ref))
print("The spectrum predicted by the model:", nn.predict([g_min_ref]))

The optimal geometry, according to AI magic: [ 70.00011854 100.00003831  19.99951484 700.00215542]
The spectrum predicted by the model: [[0.01913309 0.98825536]]


In [15]:
#Compare with best measurement in tm train dataset - real loss = 2.22e-5
print("Optimal simulated geometry [x1, x2, gap, he]:", [80,90,20,700])
g0 = scaler.transform([[80,90,20,700]])
#print(g0[0])
sp = nn.predict(g0)[0]
ssim = [0.002775338, 0.996192651]
print("Simulated [p0, p1]:", ssim)
print("Predicted [p0, p1]:", sp)
ls = pow((ssim[0]-0),2) + pow((ssim[1]-1), 2)
lp = pow((sp[0]-0),2) + pow((sp[1]-1), 2)
print("Simulated loss:", ls)
print("Predicted loss:", lp)
if(ls>=lp):
    print("Is predicted better than simulated?: YES!")
else:
    print("Is predicted better than simulated?: NO!!")

#loss([0,1], nn, g0[0], mu_g, r)

Optimal simulated geometry [x1, x2, gap, he]: [80, 90, 20, 700]
Simulated [p0, p1]: [0.002775338, 0.996192651]
Predicted [p0, p1]: [0.02121002 0.97754033]
Simulated loss: 2.2198407422045116e-05
Predicted loss: 0.0009543014932363147
Is predicted better than simulated?: NO!!


In [20]:
#Perform gradient descent for the smaller model
g_min = X_train[0]
loss_min = 1
for x in X_train:
    g = opt([0,1], nn_gs_2, x, delta, 0.01, mu_g, r, 1e-5, 1e3)
    l = loss([0,1], nn_gs_2, g, mu_g, r)
    if (l < loss_min):
        g_min = g
        loss_min = l
print(loss_min)
nn.predict([g_min])
#Present optimal CV geometry
g_min_ref = g_min
print("The optimal CV geometry, according to AI magic:", scaler.inverse_transform(g_min))
print("The spectrum predicted by the model:", nn.predict([g_min]))

1000
221
1000
674
465
1000
634
826
367
1000
1000
1000
1000
199
1000
1000
1000
1000
368
149
1000
329
1000
420
886
1000
1000
1000
795
1000
729
169
951
1000
1000
1000
253
1000
567
1000
195
1000
1000
1000
947
149
1000
1000
1000
138
1000
1000
823
1000
343
1000
644
1000
1000
901
1000
928
1000
565
1000
1000
814
1000
567
756
1000
1000
1000
1000
225
1000
341
130
1000
260
1000
698
303
1000
895
1000
1000
1
1000
1000
1000
1000
125
1000
1000
1000
403
1000
1000
1
567
1000
1000
683
1000
903
536
272
1000
1000
734
1
987
496
1000
233
977
1000
1000
1000
1000
1000
224
1000
1000
224
1000
1000
1000
1000
342
893
633
140
486
205
1
1000
62
806
1000
1000
1000
1000
1
963
1000
1000
1000
1000
385
771
1
1000
1
876
1000
1000
1000
98
611
1000
434
1000
723
1000
1000
1000
1000
1000
512
1000
1000
134
996
1
1000
1000
499
192
1000
1
91
1000
528
1000
1000
1000
1000
1
1000
1000
258
1000
896
1000
991
1000
458
1000
1000
1000
1000
795
207
1000
726
284
1000
717
1000
550
1000
1
806
426
271
1000
940
1000
1000
216
1000
255
1
1000


In [21]:
#Check gradient descent for 10000 tries - better?
nn = nn_ref
mu_g = np.mean(X_train, axis = 0)
delta = [1/10000, 1/10000, 1/10000, 1/2000]
g_min = X_train[0]
loss_min = 1
for x in X_train:
    g = opt([0,1], nn, x, delta, 0.01, mu_g, r, 1e-5, 1e4)
    l = loss([0,1], nn, g, mu_g, r)
    if (l < loss_min):
        g_min = g
        loss_min = l
print(loss_min)
nn.predict([g_min])
print("The optimal reference geometry, according to AI magic, 10000 tries:", scaler.inverse_transform(g_min))
print("The spectrum predicted by the model:", nn.predict([g_min]))

1092
2106
1382
2397
2101
1
594
975
690
490
2798
1177
1111
151
2159
2621
3915
1106
479
806
987
213
1702
506
933
1487
2653
2064
378
1761
583
935
930
2871
1397
931
333
653
584
1182
1180
2363
1185
812
989
2248
2304
950
1953
1968
2974
1861
249
1142
48
2104
424
2270
1109
1856
1233
2498
3241
244
4173
1921
1521
1530
431
574
3590
1
2678
4342
3389
1927
1805
2533
1268
1
309
1645
2012
1188
701
2936
2021
1
2681
4434
231
1107
54
644
1277
1551
169
1503
2493
2209
669
837
1673
517
694
1664
509
445
1677
2654
726
3631
1669
2097
2048
4974
1259
1742
1504
1891
1795
2425
531
3200
1463
460
437
1984
897
2292
178
2040
647
1254
589
1
5830
2409
618
500
2396
3089
1158
752
2520
656
2436
1371
1438
1333
1283
1520
1
2074
1
2084
1196
3001
789
549
659
1316
1377
4904
1106
3696
1416
2490
794
2266
154
1285
501
157
1149
1
1
342
1883
812
2219
1741
248
1
2194
2264
1914
604
1511
1
2439
2756
3009
1934
2220
995
789
1608
279
3327
1348
2603
770
1235
880
1229
805
4028
1831
1829
2368
389
2291
1
768
798
130
524
2457
1084
1804
2074
68

In [22]:
#Check gradient descent for 10000 tries - better?
g_min = X_train[0]
loss_min = 1
for x in X_train:
    g = opt([0,1], nn_gs_2, x, delta, 0.01, mu_g, r, 1e-5, 1e4)
    l = loss([0,1], nn_gs_2, g, mu_g, r)
    if (l < loss_min):
        g_min = g
        loss_min = l
print(loss_min)
nn.predict([g_min])
#Present optimal CV geometry
g_min_ref = g_min
print("The optimal CV geometry, according to AI magic, 10000 tries:", scaler.inverse_transform(g_min))
print("The spectrum predicted by the model:", nn.predict([g_min]))

1130
221
1139
674
465
2046
634
826
367
1860
2657
1304
1014
199
2437
3766
4184
1112
368
149
1151
329
1867
420
886
1906
2557
4689
795
1089
729
169
951
2724
1019
1044
253
1380
567
1340
195
1515
1285
1038
947
149
1703
1789
2149
138
2831
1371
823
2174
343
1178
644
1700
1121
901
1139
928
1171
565
2583
2560
814
1328
567
756
1371
1674
1125
2783
225
2096
341
130
1970
260
1536
698
303
1211
895
1268
2392
1
3135
2409
1047
1766
125
3826
1366
1213
403
1271
1693
1
567
1143
1240
683
1493
903
536
272
1480
1012
734
1
987
496
1812
233
977
3587
2550
2091
3716
3916
224
3127
1347
224
1741
1718
1082
1182
342
893
633
140
486
205
1
2910
62
806
2190
1070
1487
2283
1
963
1918
1257
1521
1222
385
771
1
1272
1
876
1105
1836
2054
98
611
2828
434
2123
723
1843
1594
1094
1200
1058
512
1275
1916
134
996
1
2329
1417
499
192
1646
1
91
1100
528
1415
1108
1066
2797
1
2351
2183
258
3438
896
1731
991
1177
458
2182
1332
2051
3258
795
207
1072
726
284
4428
717
1351
550
3105
1
806
426
271
1093
940
1968
1343
216
1363
255
1
1005
